## 5.1 Insertion

In [8]:
// my solution is 5% slower b/c I generate the sequence of 1s using Math.pow when i could've just done ~0 or -1
// otherwise, i basically just combined the 2 steps of making the left mask and right mask into one-line

function insertion(N, M, i, j) {
    let pow = Math.pow(2, j - 1);  // turned it into sequence of 1s 11111
    let mask = ~( ( pow - 1 ) << i); // shifted 1s by i then negated
    N &= mask;
    return N | (M << i);
}
var N = 0b10000000000;
var M = 0b10011;
insertion(N, M, 2, 6).toString(2);

'10001001100'

### Book Solutions:
* M = inserted sequence
* N = larger sequence getting M inserted into it
* solution:
    1. clear the bits j through i in N 
        - this can be done by creating a mask where the bits from j through i are 0 and the surrounding bits are 1s
        - __if the mask seems complex, separate it into parts__
        - so for our mask, we needed something like this: 111100000011;
        - this can be split into the left side: 11110000000
        - and the right side: 00000000011
        - then you can combine them together
    2. shift M so that it lines up with bits j through i
        - from the prompt, we know that bits j through i have enough space to fill all of m, so all you need to do is just shift M by i
        - so M << i
        - this would automatically put the sequence at pos j through i in M
    3. merge M and N
        - then you do M | N to merge the 2 together

In [43]:
function updateBits(n, m, i, j) {
    // we want the mask to look like 11110000011; where the 0s clear out the bit where M is supposed to be
    let allOnes = ~0; //sequence of 1s
    
    // this would create 11110000000
    let left = allOnes << (j + 1);
    
    // this would create 00000000011;
    let right = ((1 << i) - 1);
    
    // left | right = 111100000011;
    let mask = left | right;
    
    let n_cleared = n & mask;
    let m_shifted = m << i;
    
    return n_cleared | m_shifted;
}

var N = 0b10000000000;
var M = 0b10011;

updateBits(N, M, 2, 6).toString(2);

'10001001100'

## 5.2 Binary To String

In [19]:
// my solution is ~45% slower b/c i generate the entire string first, then i check whether it is over 32 characters
// plus i cheated since i used javascript's toString without doing it myself

function binString(num) {
    let bin = num.toString(2);
    console.log(bin)
    let length = bin.slice(2).length;
    if(length > 32) {
        console.log('ERROR');
    }
    else {
        console.log(bin)
    }
}

binString(0.72)

0.1011100001010001111010111000010100011110101110000101
ERROR


### Book Solutions:
* solution:
    1. create a temp variable to hold the current value called r
    2. r = num * 2;
    3. if r >= 1, then you add 1 to the binary string
        - and you set num as equal to the decimal part of the number only (that's why we subtract by 1)
        - ex: if your number was 1.25, 1.25 - 1 = 0.25, i.e. the decimal portion
    4. if r < 1, then you concat 0 and you still use the decimal portion of the number!
    5. if the string so far is about 32 chars long, return 'ERROR', else return the binary string once it is done

In [18]:
function printBinary(num) {
    if(num >= 1 || num <= 0) {
        return 'ERROR';
    }
    
    let binary = '.';
    
    while(num > 0) {
        if(binary.length >= 32) {
            console.log(binary)
            return 'ERROR';
        }
        
        let r = num * 2;
        
        if(r >= 1) {
            binary = binary.concat('1');
            num = r - 1;
        }
        else {
            binary = binary.concat('0');
            num = r;
        }
    }
    return binary;
}

printBinary(0.72);

.1011100001010001111010111000010


'ERROR'

## 5.3 Flip Bit to Win

In [21]:
// my solution is 13% slower b/c i generate the num as a binary string then go through it

function flipToWin(num) {
    let before0 = 0;
    let after0 = 0;
    num = num.toString(2);
    for(let i = 0; i < num.length; i++) {
        let bit = num[i];
        if(bit === '1') {
            after0++;
        }
        if(bit === '0') {
            before0 = after0;
            after0 = 0;
        }
    }
    return {count: before0 + after0 + 1}
}

console.log(flipToWin(0b11011101111));
console.log(flipToWin(0b100000101));

{ count: 8 }
{ count: 3 }


### Book Solutions
* solution:
    1. check if the number is already just a sequence of 1s, for example 0b1111111.
    2. have 3 variables
        - currentLength = current length of 1s as we iterate through bits
        - previousLength = length of 1s before we encountered a 0
        - maxLength = max length of 1s seq we can make so far
    3. iterate through the entire bit sequence of the number
        - a & 1 = compares rightmost bit to 1
        - ex: a = 0b00011, a & 1 = 0b00011 & 0b00001
        - a & 2 = compares next bit in line:
        - ex: a = 0b00010, a & 2 = 0b00110 & 0b00010
            - we do this b/c if the sequences are not separated by a 0 in the middle: like so : 0b11100111, then prev = 0
            - else if it is: 0b1110111, then we set it to the currentLength
    4. then at the end, we compare maxLength to the current and previous lengths + 1
    5. then we do a logical right shift for a: a >>>= 1
        - this is to ensure we escape the while loop (while a !== 0)
        - and to ensure we move the bits to the right to check
        - ex: 0b000110 >>> 1 = 0b000011

In [25]:
function flipBit(a) {
    if(~a === 0) {
        // the whole number is a sequence of 1s so just return how many bytes are in this
        // since flipping a bit wouldn't make it any longer
        return a.toString(2).length;
    }
    
    let currentLength = 0; // current length of 1s sequence after a 0
    let previousLength = 0; // previous length of 1s sequence before the same 0
    let maxLength = 1; //maxLength default to 1 b/c you can flip a 0-bit in 000000 and would have 1
    
    // iterates through the numbers bits from right to left
    while(a !== 0) {
        // if the rightmost bit is currently a 1, increment currentLength
        if ((a & 1) === 1) {
            currentLength++;
        }
        // else if the bit is current a 0
        else if ((a & 1) === 0) {
            
            // if the bit after the current bit = 0, then previousLength = 0
            // else, we set it to the currentLength
            previousLength = (a & 2) === 0 ? 0 : currentLength;
            
            // then reset currentLength to count the sequence after the 0
            currentLength = 0;
        }
        
        // updates the maxLength
        maxLength = Math.max(previousLength + currentLength + 1, maxLength);
        
        // THIS CODE IS IMPORTANT B/C WE ARE ABLE TO ITERATE THROUGH A BY DOING THIS
        // WE MOVE ALL BITS TO THE RIGHT SO THE NEXT BIT WE CHECK IS THE BIT AFTER THE CURRENT ONE
        
        // IN ADDITION, THIS CODE IS IMPORTANT TO ESCAPE THE WHILE LOOP.
        // REMEMBER THAT LOGICAL RIGHT SHIFTING SETS THE LEFTMOST BIT TO 0 SO AFTER A WHILE
        // THE NUMBER B/C 0
        a >>>= 1;
    }
    return maxLength;
}

console.log(flipBit(0b11011101111));
console.log(flipBit(0b100000101));

8
3


## 5.4 Next Number

In [125]:
// failed to finish this

function nextLargest(n) {
    let found = false;
    while( !found ) {
        n++;
        let count = 0;
        let x = n;
        while(x !== 0) {
            x &= (x - 1)
            count++;
        }
    }
}

function nextSmallest(n) {
    
}

function nextNum(n) {
    let nextL = nextLargest(n);
    let nextS = nextSmallest(n);
    return {nextL, nextS}
}

console.log(nextNum(11).toString(2))

1011
1011


In [132]:
var x = 15;
console.log(x.toString(2))
var count = 0;
while(x !== 0) {
    x &= (x - 1);
    count++;
}
console.log(count)

1111
4


### Book Solutions:
* brute force approach:
    1. count the number of 1s in n
    2. increment and decrement number and check number of 1s in that
    3. if they match the number of 1s in n, then we have the next biggest and smallest number
* bit manipulation approach to get next largest num:
    1. flip rightmost non-trailing zero to a 1
        - ex: 11011001111100, the rightmost non-trailing zero is at position 7 or 110110 __0__ 1111100
        - so now we have 110110 __1__ 1111100
        - and we call that p = position 7
    2. then we count the number of 1s and 0s to the right of p
        - c1 = number of 1s = 5
        - c0 = number of 0s = 2
        - p = 7 (position 7)
    3. we clear the bits to the right of p so from p - 1 to 0
        - mask = ~( ( 1 << p ) - 1 );
        - this essentially makes 11111110000000
        - 1 << p shifts 1 to pos p
        - then when we subtract by 1, we create a bunch of 1s from p to 0, so 00000000111111
        - then we negate that ~(0000000011111) to get 11111110000000
    4. then we add in c1 - 1 1s
        - so we add in (c1 - 1) = 5 - 1 = 4 1s or 1111
        - we create a mask: (1 << (c1 - 1)) - 1
        - we shift 1 by c1 -1 to create something like this: 00000100000
        - then we subtract by 1, which creates a bunch of 1s from c1 - 1 to 0
        - so we have 0000001111111
    5. so now we have 110110 __1__ 0001111
* bit manipulation approach to get next smallest num: (symmetric to largest)
    1. get positiion of the rightmost non-trailing 1 and flip it to a 0
        - ex: 10011110000011
        - rightmost non-trailing 1 is at pos 7: 100111 __1__ 0000011
        - so now we have: 100111 __0__ 0000011 after flipping it
    2. get the number of 1s and 0s to the right of that non-trailing 1
        - p = 7 (pos 7)
        - c1 = 2
        - c0 = 5
    3. clear all bits to the right of that position
        - after clearing those bits to right of position 7, we have 100111 __1__ 0000000
        - mask = ~0 << (p + 1) which creates 1111100000
    4. insert c1 + 1 ones to the __IMMEDIATE right__ of p, NOT AT THE RIGHTMOST PORTION OF THE NUMBER!!!
        - we can do this by doing:
        - a = 1 << (c1 + 1) to create 00000100000
        - b = a - 1 to create 00000011111
        - c = b << (c0 - 1) = 001111110000 since c0 = 5 then - 1
        - then we would have 100111 __0__ 1110000
            - as you can see, the 1s are immediately before the the bolded __0__ instead of all the way to the right of the bit sequence
* arithmetic approach to get next largest num: (basically the bit approach but with math)
    1. set the pth bit to 1
    2. set all bits following p to 0
    3. set bits 0 through c1 - 2 to 1 (this will be c1 - 1 total bits)
* arithmtic approach to get next smallest num:
    1. set pth bit to 0
    2. set all bits following p to 1
    3. set bits 0 through c0 - 1 to 0

In [39]:
function getNextLargest(n) {
    let c = n;
    let c0 = 0;
    let c1 = 0;
    
    // computes c0;
    // if there are no more trailing 0s, then stop
    while( ( (c & 1) === 0 ) && (c !== 0 ) ) {
        c0++;
        c >>= 1;
    }
    
    // computes c1;
    while( (c & 1) === 1) {
        c1++;
        c >>= 1;
    }
    
    // 31 = 011111
    // if there are no trailing 0s, cannot have a number larger than this with the same number of 1s
    if( (c0 + c1 === 31) || (c0 + c1 == 0)) {
        return -1;
    }
    
    let p = c0 + c1;
    
    n |= (1 << p);
    n &= ~( (1 << p) - 1 );
    n |= (1 << (c1 - 1)) - 1;
    return n;
}

function getNextSmallest(n) {
    let temp = n;
    let c0 = 0;
    let c1 = 0;
    
    // calculates number of trailing 1s
    while(temp & 1 === 1) {
        c1++;
        temp >>= 1;
    }
    
    // if no trailing 1s, then cannot find smallest number with same number of 1s
    if(temp === 0) {
        return -1;
    }
    
    while( (temp & 1) === 0 && (temp !== 0)) {
        c0++;
        temp >>= 1;
    }
    
    let p = c0 + c1;
    n &= ( ~0 << (p + 1));
    
    let mask = (1 << (c1 + 1)) - 1;
    n |= mask << (c0 - 1);
    
    return n;
}

function getNext(num) {
    let nextLargest = getNextLargest(num);
    let nextSmallest = getNextSmallest(num);
    return {nextLargest,nextSmallest}
}

getNext(11)

{ nextLargest: 13, nextSmallest: 7 }

## 5.5 Debugger

In [146]:
// determines if the number is a power of 2
function debug(n) {
    return ( n & (n - 1) ) === 0
}

console.log(debug(8))
console.log(debug(6))

true
false


### Book Solutions:
* checks for power of 2
* n - 1 = removes the rightmost 1 bit from n and then makes any trailing 0s before it = 1
* then when we and it together, it removes every 1 bit from n
* if it does that, then n & (n - 1) = 0
* this therefore checks for a power of 2 b/c if it had more than 1 1-bit, then it would not be a power of 2 and would therefore have a non-zero number

## 5.6 Conversion

In [49]:
function conversion(a, b) {
    let convert = a ^ b;
    let count = 0;
    while(convert !== 0) {
        convert &= convert - 1
        count++;
    }
    return count;
}

conversion(0b11101, 0b01111);

2

### Book Solutions:
* solution:
    1. XOR the 2 numbers and the spots where it has a 1-bit are the ones where they differ
    2. count the number of 1s in it and that will be the number of swaps needed to make them the same

In [53]:
function bitSwapRequired(a, b) {
    let count = 0;
    for(let c = a ^ b; c !== 0; c = c & (c - 1)) {
        count ++;
    }
    return count;
}

bitSwapRequired(0b11101, 0b01111);

2

## 5.7 Pairwise Swap

In [59]:
function pairSwap(n) {
    let odd = n & 0xaaaaaaaa; //1010
    let even = n & 0x55555555; // 0101
    odd >>>= 1;
    even <<= 1;
    return odd | even;
}

pairSwap(0b11011011).toString(2);

'11100111'

### Book Solutions:
* solution:
    1. mask all odd bits with 0xa = 1010
        - odd = num & 0xaaaaaaaa
    2. mask all even bits with 0x5 = 0101
        - even = num & 0x55555555
    3. shift the odd bits to the right to make put them in the even position
        - odd >>>= 1;
        - use of logical right shift b/c want sign to be filled with a zero
    4. shift all the even bits to the right to put them in the odd position
        - even <<= 1;

In [63]:
function swapOddEvenBits(x) {
    return ( (x & 0xaaaaaaaa) >>> 1 | (x & 0x55555555) << 1 ) ;
}

swapOddEvenBits(0b11011011).toString(2);

'11100111'

## 5.8 Draw Line

In [80]:
function getHeight(row, offset) {
    let index = 0;
    for(let i = 1; i < row; i++) {
        index += offset;
    }
    return index;
}

function drawLine(screen, width, x1, x2, y) {
    let rowIndex = getHeight(y, width / 8);
    
    while(rowIndex < (rowIndex + (width / 8) )) {
        let mask = 0b11111111; // 255 (1 byte)
        if(x1 !== 0) {
            mask >>= x1;
            if(x2 < 7) {
                mask >>= 1;
                mask <<= 7 - x2;
            }   
        }
        else if (x2 < 7) {
            mask >>= x2;
            mask = ~(mask);
        }
        screen[rowIndex] |= mask;
        rowIndex++;
        x1 = 0;
        x2 -= 7;
        if(x2 < 0) {
            break;
        }
    }
    return screen;
}

function printResult(screen, width, x1, x2, y) {
    return drawLine(screen, width, x1, x2, y).map(el => {
        let binRep = (el >>> 0).toString(2);
        if(binRep.length <= 8) {
            while(binRep.length < 8) {
                binRep = '0' + binRep;
            }
            return binRep;
        }
        else{
            return binRep.slice(32-8);
        } 
    })
}

var screen1 = [0b00000000, 0b00000000, 0b00000000, 0b00000000, 0b00000000, 0b0000000];
var screen2 = [0b00000000, 0b00000000, 0b00000000, 0b00000000, 0b00000000, 0b0000000];
var screen3 = [0b00000000, 0b00000000, 0b00000000, 0b00000000, 0b00000000, 0b0000000];
var width = 16; // so 0b00000000 and 0b00000000 = 1 row;
console.log(printResult(screen1, width, 5, 10, 2));
console.log('\n');
console.log(printResult(screen2, width, 2, 9, 3));
console.log('\n');
console.log(printResult(screen3, width, 2, 6, 3));

[ '00000000',
  '00000000',
  '00000111',
  '11100000',
  '00000000',
  '00000000' ]


[ '00000000',
  '00000000',
  '00000000',
  '00000000',
  '00111111',
  '11000000' ]


[ '00000000',
  '00000000',
  '00000000',
  '00000000',
  '00111110',
  '00000000' ]


### Book Solutions
* solution: 
    1. instead of iterating through an entire byte and setting each one, you can use masks
    2. if the line occupies a whole byte (0 - 7), then you can just use a mask and set every bit using 0xFF
    3. if the line occupies a small portion of the byte or it spills over, you can create masks for the off-set at the start of it and off_set at the beginning of it

In [73]:
function drawLineSoln(screen, width, x1, x2, y) {
    let start_offset = x1 % 8;
    let first_full_byte = x1 / 8;
    if(start_offset !== 0) {
        first_full_byte++;
    }
    
    let end_offset = x2 % 8;
    let last_full_byte = x2 / 8;
    if(end_offset !== 7) {
        last_full_byte--;
    }
    
    //set full bytes
    
    for(let b = first_full_byte; b <= last_full_byte; b++) {
        screen[(width / 8) * y + b] = 0xff;
    }
    
    // create masks for start and end of line
    let start_mask = (0xFF >> start_offset);
    let end_mask = ~(0xFF >> (end_offset + 1));
    
    // set start and end of line
    if((x1 / 8) === (x2 / 8)) {
        // x1 and x2 in same byte
        let mask = start_mask & end_mask;
        screen[(width / 8) * y + (x1 / 8)] |= mask;
    }
    else {
        if(start_offset !== 0) {
            let byte_number = (width / 8) * y + first_full_byte - 1;
            screen[byte_number] |= start_mask;
        }
        if(end_offset !== 7) {
            let byte_number = (width / 8) * y + last_full_byte + 1;
            screen[byte_number] |= end_mask;
        }
    }
    return screen;
}

function printResultSoln(screen, width, x1, x2, y) {
    return drawLineSoln(screen, width, x1, x2, y).map(el => {
        let binRep = (el >>> 0).toString(2);
        if(binRep.length <= 8) {
            while(binRep.length < 8) {
                binRep = '0' + binRep;
            }
            return binRep;
        }
        else{
            return binRep.slice(32-8);
        } 
    })
}

var screen1 = [0b00000000, 0b00000000, 0b00000000, 0b00000000, 0b00000000, 0b0000000];
var width = 16;
console.log(drawLineSoln(screen1, width, 5, 10, 2));

[ 0, 0, 0, 0, 0, 0, '4.625': 7, '5.25': -32 ]
